In [1]:
import itertools
stocks = ['RELIANCE.NS', 'TATASTEEL.NS', 'HDFCBANK.NS', 'INFY.NS', 'BAJAJ-AUTO.NS', 'ICICIBANK.NS', 'ITC.NS',  'UPL.NS', 'ONGC.NS', 'HINDALCO.NS', 'TITAN.NS', 'COALINDIA.NS', 'INDUSINDBK.NS', 'BAJAJFINSV.NS', 'GRASIM.NS', 'JSWSTEEL.NS']
crypto = ['BTC-USD', 'ETH-USD', 'ADA-USD', 'DOGE-USD', 'SOL1-USD', 'LTC-USD', 'BNB-USD', 'AVAX-USD', 'UNI3-USD', 'DOT1-USD', 'SUSHI-USD', 'LINK-USD', 'XRP-USD', 'ALGO-USD', 'EOS-USD', 'XTZ-USD', 'FIL-USD', 'ATOM1-USD', 'TRX-USD', 'MATIC-USD', 'CHZ-USD', 'FTT1-USD', 'NEAR-USD', 'SAND-USD', 'WAVES-USD', 'IOST-USD', 'DASH-USD', 'STORJ-USD', 'ZEC-USD', 'KSM-USD']
commodity = ['GC=F', 'SI=F', 'HG=F', 'PL=F', 'PA=F', 'CL=F', 'NG=F', 'RB=F', 'HO=F', 'BZ=F', 'KC=F', 'SB=F', 'CT=F', 'HE=F', 'LE=F', 'GOLD']


In [2]:
import yfinance as yf
import numpy as np
import pandas as pd

# # Download historical stock data for a specific ticker
# ticker = "AAPL"
# data = yf.download(ticker, start="2018-01-01", end="2023-05-10")

# Define functions to calculate technical indicators
def MACD(df, fast=12, slow=26, signal=9):
    exp1 = df["Close"].ewm(span=fast, adjust=False).mean()
    exp2 = df["Close"].ewm(span=slow, adjust=False).mean()
    macd = exp1 - exp2
    signal = macd.ewm(span=signal, adjust=False).mean()
    return macd, signal

def ATR(df, n=14):
    high = df["High"]
    low = df["Low"]
    close = df["Close"]
    tr1 = high - low
    tr2 = abs(high - close.shift())
    tr3 = abs(low - close.shift())
    true_range = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = true_range.rolling(n).mean()
    return atr

def normalized_ATR(df, n=14):
    atr = ATR(df, n)
    natr = atr / df["Close"] * 100
    return natr

def momentum(df, n=10):
    return df["Close"].diff(n)

def CO(df):
    adl = (2*df["Close"]-df["High"]-df["Low"])/(df["High"]-df["Low"])*df["Volume"]
    ema3 = adl.ewm(span=3, adjust=False).mean()
    ema10 = adl.ewm(span=10, adjust=False).mean()
    co = ema3 - ema10
    return co

def OBV(df):
    vol = df["Volume"]
    change = np.where(df["Close"].diff() > 0, 1, -1)
    obv = (vol * change).cumsum()
    return obv

# def MFI(df, n=14):
#     typical_price = (df["High"] + df["Low"] + df["Close"]) / 3
#     money_flow = typical_price * df["Volume"]
#     pos_flow = np.where(typical_price.diff() > 0, money_flow, 0)
#     neg_flow = np.where(typical_price.diff() < 0, money_flow, 0)
#     pos_mf = pos_flow.rolling(n).sum()
#     neg_mf = neg_flow.rolling(n).sum()
#     mf_ratio = pos_mf / neg_mf
#     mfi = 100 - (100 / (1 + mf_ratio))
#     return mfi
# def MFI(df, n=14):
#     # Calculate the typical price
#     typical_price = (df["High"] + df["Low"] + df["Close"]) / 3
#     # Calculate the raw money flow
#     raw_money_flow = typical_price * df["Volume"]
#     # Calculate the money flow ratio
#     positive_flow = np.where(typical_price > typical_price.shift(), raw_money_flow, 0)
#     negative_flow = np.where(typical_price < typical_price.shift(), raw_money_flow, 0)
#     positive_mf = pd.Series(positive_flow).rolling(window=n, min_periods=0).sum()
#     negative_mf = pd.Series(negative_flow).rolling(window=n, min_periods=0).sum()
#     money_flow_ratio = positive_mf / negative_mf
#     # Calculate the money flow index
#     money_flow_index = 100 - (100 / (1 + money_flow_ratio))
#     return money_flow_index


def MFI(df):
    # Calculate the Money Flow Index
    typical_price = (df['High'] + df['Low'] + df['Close']) / 3
    raw_money_flow = typical_price * df['Volume']
    up_flow = np.where(typical_price > typical_price.shift(1), raw_money_flow, 0)
    down_flow = np.where(typical_price < typical_price.shift(1), raw_money_flow, 0)
    positive_money_flow = np.sum(up_flow)
    negative_money_flow = np.sum(down_flow)
    money_ratio = np.divide(positive_money_flow, negative_money_flow, where=negative_money_flow!=0)
    mfi = np.where(negative_money_flow==0, 100, 100 - (100 / (1 + money_ratio)))
    return mfi



# def HTDCP(df):
#     cycle, trend = pd.core.window.HammingWindow(window_len=11).frequency_response()*100
#     hilbert = pd.Series(pd.Series.rolling(df["Close"], window=31, center=True).apply(lambda stocks: np.abs(pd.Series(x).hilbert().real))).ewm(span=21, adjust=False).mean()
#     phase = np.degrees(np.arctan((hilbert - hilbert.shift(1)) / 0.0001))
#     dcp = ((phase.rolling(14).sum() / 14) - 90) * (-1)
#     return dcp
from scipy.signal import hilbert
from scipy.signal.windows import hamming

def HTDCP(df):
    # Calculate the Hilbert Transform Dominant Cycle Phase
    analytic_signal = hilbert(df["Close"])
    instantaneous_phase = np.degrees(np.unwrap(np.angle(analytic_signal)))
    hilbert_transform = np.abs(analytic_signal)
    hilbert_envelope = pd.Series(hilbert_transform).rolling(window=31, center=True, min_periods=1).mean()
    hilbert_smoothed = pd.Series(hilbert_envelope).rolling(window=11, center=True, min_periods=1).apply(lambda stocks: np.convolve(x, hamming(11), mode='same'), raw=True)
    hilbert_smoothed /= np.sum(hamming(11))
    hilbert_difference = np.abs(hilbert_smoothed.diff())
    hilbert_smoothed_numpy = np.asarray(hilbert_smoothed)  # convert to numpy array
    dcphase = np.degrees(np.unwrap(np.angle(pd.Series(hilbert_smoothed_numpy * np.exp(-1j * instantaneous_phase)).rolling(window=31, center=True).mean())))
    return dcphase




# def HTS(df, n=14):
#     # Extract the analytic signal using the Hilbert Transform
#     hilbert = pd.Series(pd.Series.rolling(df["Close"], window=31, center=True).apply(lambda x: pd.Series(x).hilbert()))
#     # Calculate the instantaneous amplitude and phase
#     inst_amplitude = np.abs(hilbert)
#     inst_phase = np.degrees(np.unwrap(np.angle(hilbert)))
#     # Calculate the sine wave
#     sinewave = np.sin(np.radians(inst_phase))
#     # Smooth the sine wave using an EMA
#     hts = pd.Series(sinewave).ewm(span=n, min_periods=n).mean()
#     return hts
from scipy.signal import hilbert

def HTS(df):
    # Calculate the Hilbert Transform Sinewave
    analytic_signal = hilbert(df['Close'])
    amplitude_envelope = np.abs(analytic_signal)
    instant_phase = np.unwrap(np.angle(analytic_signal))
    sinewave = np.sin(instant_phase)
    return sinewave


# def HTTMM(df):
#     # Calculate the Hilbert Transform Trend Market Mode
#     imf = pd.core.window.HammingWindow(window_len=11).frequency_response()*100
#     hilbert = pd.Series(pd.Series.rolling(df["Close"], window=31, center=True).apply(lambda x: np.abs(pd.Series(x).hilbert().real))).ewm(span=21, adjust=False).mean()
#     smoothed_imf = pd.Series(hilbert).ewm(span=14, adjust=False).mean()
#     trend = np.where(smoothed_imf > smoothed_imf.shift(), 1, -1)
#     return trend


# from scipy.signal import hilbert, chirp, find_peaks, peak_widths, welch, windows

# def HTTMM(df):
#     # Calculate the Hilbert Transform Trend Market Mode
#     instantaneous_phase = np.unwrap(np.angle(hilbert(df['Close'])))
#     inst_period = np.diff(instantaneous_phase)
#     inst_period = np.insert(inst_period, 0, inst_period[0])
#     inst_frequency = np.divide(1, inst_period)
#     frequency = welch(df['Close'], window='hamming', nperseg=len(df['Close']))[0]
#     peak_ind, _ = find_peaks(frequency, prominence=0.1)
#     widths = peak_widths(frequency, peak_ind, rel_height=0.5)
#     dominant_period = np.mean(widths[0] / len(frequency))
#     trend_market_mode = np.mod(np.divide(360, dominant_period) * instantaneous_phase, 360)
#     return trend_market_mode


from scipy.signal import hilbert, chirp, find_peaks, peak_widths, welch, windows

def HTTMM(df):
    if df.empty:
        return pd.Series()
    
    # Calculate the Hilbert Transform Trend Market Mode
    instantaneous_phase = np.unwrap(np.angle(hilbert(df['Close'])))
    inst_period = np.diff(instantaneous_phase)
    inst_period = np.insert(inst_period, 0, inst_period[0])
    inst_frequency = np.divide(1, inst_period)
    frequency = welch(df['Close'], window='hamming', nperseg=len(df['Close']))[0]
    peak_ind, _ = find_peaks(frequency, prominence=0.1)
    widths = peak_widths(frequency, peak_ind, rel_height=0.5)
    dominant_period = np.mean(widths[0] / len(frequency))
    trend_market_mode = np.mod(np.divide(360, dominant_period) * instantaneous_phase, 360)
    return trend_market_mode

# # Create a new dataframe to store the technical indicators
# features = pd.DataFrame(index=data.index)

# # Calculate the technical indicators
# features["macd"], features["signal"] = MACD(data)
# features["atr"] = ATR(data)
# features["natr"] = normalized_ATR(data)
# features["momentum"] = momentum(data)
# features["co"] = CO(data)
# features["obv"] = OBV(data)
# features["mfi"] = MFI(data)
# features["dcp"] = HTDCP(data)
# features["hts"] = HTS(data)
# features["httmm"] = HTTMM(data)

# # Print the resulting dataframe
# print(features.head())


In [3]:
from ta import add_all_ta_features

In [4]:

import yfinance as yf
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler


def generate_data(stocks):
  data_stock = {}
  for stock in stocks:
      data_stock[stock] = yf.download(stock, period='2y',interval='1d')
  # rate of return over 2 days
  for stock in stocks:
      data_stock[stock]['return'] = (data_stock[stock]['Close'] / data_stock[stock]['Open']) - 1
      data_stock[stock]['return']*=100
      #new
  for stock in stocks:
      data_stock[stock]['macd'], data_stock[stock]['signal'] = MACD(data_stock[stock])

  for stock in stocks:
      data_stock[stock]["atr"] = ATR(data_stock[stock])

      data_stock[stock]["natr"] = normalized_ATR(data_stock[stock])
      data_stock[stock]["momentum"] = momentum(data_stock[stock])
      data_stock[stock]["co"] = CO(data_stock[stock])
      data_stock[stock]["obv"] = OBV(data_stock[stock])
      data_stock[stock]["mfi"] = MFI(data_stock[stock])
      # data_stock[stock]["dcp"] = HTDCP(data_stock[stock])
      data_stock[stock]["hts"] = HTS(data_stock[stock])
      # data_stock[stock]["httmm"] = HTTMM(data_stock[stock])

  # movig avg 10 and 5 days
  for stock in stocks:
    data_stock[stock]['ma_10d'] = data_stock[stock]['Close'].rolling(window=10, min_periods=1).mean()
    data_stock[stock]['ma_5d'] = data_stock[stock]['Close'].rolling(window=5, min_periods=1).mean()
      
  # volatility
  for stock in stocks:
    data_stock[stock]['volatility'] = np.log(data_stock[stock]['Close'] / data_stock[stock]['Close'].shift(1)).rolling(window=10).std() * np.sqrt(252)
  # volume
  for stock in stocks:
    data_stock[stock]['volume'] = data_stock[stock]['Volume']
  # rsi
  for stock in stocks:
    delta = data_stock[stock]['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14, min_periods=1).mean()
    avg_loss = loss.rolling(window=14, min_periods=1).mean()
    rs = avg_gain / avg_loss
    data_stock[stock]['rsi'] = 100 - (100 / (1 + rs))
      
  # psychological index
  for stock in stocks:
    n = 10
    data_stock[stock]['up_days'] = data_stock[stock]['Close'] > data_stock[stock]['Close'].shift(1)
    data_stock[stock]['Nup'] = data_stock[stock]['up_days'].rolling(window=n).sum()
    data_stock[stock]['N'] = n
    data_stock[stock]['psychological_index'] = data_stock[stock]['Nup'] / data_stock[stock]['N']
  for stock in stocks:
    data_stock[stock]['return_prev_day'] = data_stock[stock]['return'].shift(1)
    data_stock[stock]['return_prev_week'] = data_stock[stock]['return'].shift(5)
    data_stock[stock]['return_prev_fortnight'] = data_stock[stock]['return'].shift(10)
  for stock in stocks:
    data_stock[stock] = add_all_ta_features(
       data_stock[stock], open="Open", high="High", 
       low="Low", close="Close", volume="Volume"
      )
  #concatenate
  df_stock = pd.concat(data_stock, keys=stocks)

  return df_stock

data = generate_data(stocks)
   

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/opt/homebrew/lib/python3.11/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/opt/homebrew/lib/python3.11/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)
/opt/homebrew/lib/python3.11/site-packages/ta/wrapper.py:483: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{colprefix}momentum_ppo"] = indicator_ppo.ppo()
/opt/homebrew/lib/python3.11/site-packages/ta/wrapper.py:484: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. T

In [5]:
# data.loc['RELIANCE.NS'].describe()

In [6]:
# data.loc['BAJAJ-AUTO.NS'].describe()

In [7]:
df_filled = data.groupby(level=0).apply(lambda x: x.fillna(x.mean()))
df_filled = df_filled.fillna(0)
# features = [
#     'return_2d', 'ma_10d', 'ma_5d', 'volatility', 'rsi', 'volume','return_prev_day','return_prev_week',
#     'return_prev_fortnight','psychological_index','atr','momentum','co','obv','mfi','hts'
# ]
features = data.columns
# remove open, high, low, close, volume
features = features.drop(['Open','High','Low','Close','Volume','Adj Close'])
data_stock = df_filled[features]
data_stock = data_stock.reset_index(level=0, drop=True)
data_stock

return       macd     signal        atr      natr  \
              Date                                                              
BAJAJ-AUTO.NS 2021-08-03 -0.046999   0.000000   0.000000  81.915661  2.157839   
              2021-08-04 -0.078227   0.303138   0.060628  81.915661  2.157839   
              2021-08-05 -0.605154  -0.519796  -0.055457  81.915661  2.157839   
              2021-08-06  0.387428   1.162755   0.188185  81.915661  2.157839   
              2021-08-09 -0.077757   3.050062   0.760561  81.915661  2.157839   
...                            ...        ...        ...        ...       ...   
UPL.NS        2023-07-28 -0.453818 -13.106796 -12.293273  10.264282  1.641891   
              2023-07-31 -0.047998 -13.033696 -12.441358   9.553571  1.529305   
              2023-08-01  0.756846 -12.748120 -12.502710  10.167855  1.625037   
              2023-08-02 -1.534663 -12.901607 -12.582489   9.885712  1.596659   
              2023-08-03 -1.967345 -14.366571 -12.939306  11.310717  1.883707   

                           momentum            co       obv        mfi  \
              Date                                                       
BAJAJ-AUTO.NS 2021-08-03  21.867829  0.000000e+00   -408981  57.014797   
              2021-08-04  21.867829 -9.123246e+04    -45026  57.014797   
              2021-08-05  21.867829 -1.618915e+05   -378844  57.014797   
              2021-08-06  21.867829 -3.877117e+04    125167  57.014797   
              2021-08-09  21.867829 -9.287989e+04    521265  57.014797   
...                             ...           ...       ...        ...   
UPL.NS        2023-07-28 -14.399963  1.680294e+05  66589202  53.163926   
              2023-07-31 -19.099976 -4.394636e+04  61331992  53.163926   
              2023-08-01 -14.250000  1.991959e+06  68897756  53.163926   
              2023-08-02 -20.000000  6.401936e+05  65920462  53.163926   
              2023-08-03 -38.950012  1.867628e+05  59208738  53.163926   

                               hts  ...  momentum_ppo  momentum_ppo_signal  \
              Date                  ...                                      
BAJAJ-AUTO.NS 2021-08-03  0.437997  ...      0.334985             0.334935   
              2021-08-04  0.303570  ...      0.334985             0.334935   
              2021-08-05  0.308358  ...      0.334985             0.334935   
              2021-08-06  0.253365  ...      0.334985             0.334935   
              2021-08-09  0.270478  ...      0.334985             0.334935   
...                            ...  ...           ...                  ...   
UPL.NS        2023-07-28 -0.136375  ...     -2.019347            -1.875296   
              2023-07-31 -0.130526  ...     -2.013683            -1.902973   
              2023-08-01 -0.188652  ...     -1.974433            -1.917265   
              2023-08-02 -0.169264  ...     -2.004301            -1.934672   
              2023-08-03 -0.359435  ...     -2.243050            -1.996348   

                          momentum_ppo_hist  momentum_pvo  \
              Date                                          
BAJAJ-AUTO.NS 2021-08-03           0.010505     -1.145273   
              2021-08-04           0.010505     -1.145273   
              2021-08-05           0.010505     -1.145273   
              2021-08-06           0.010505     -1.145273   
              2021-08-09           0.010505     -1.145273   
...                                     ...           ...   
UPL.NS        2023-07-28          -0.144051     -1.243968   
              2023-07-31          -0.110710      8.560818   
              2023-08-01          -0.057168     20.370557   
              2023-08-02          -0.069629     17.479521   
              2023-08-03          -0.246702     23.121927   

                          momentum_pvo_signal  momentum_pvo_hist  \
              Date                                                 
BAJAJ-AUTO.NS 2021-08-03            -1.216227          -0.109326  

In [8]:
data_stock.loc['RELIANCE.NS'].describe().T

,count,mean,std,min,25%,50%,75%,max
return,498.0,-0.023372,1.401885,-6.454618,-0.883882,0.005962,0.787628,5.281757
macd,498.0,7.137308,37.266597,-60.507392,-22.765438,4.463061,32.334598,93.166771
signal,498.0,7.176838,34.127808,-53.997494,-19.629632,5.684578,30.333335,87.543320
atr,498.0,53.388520,13.347621,28.250017,44.553563,51.191040,60.878540,86.892857
natr,498.0,2.144393,0.528264,1.131608,1.802268,2.059242,2.497299,3.504310
...,...,...,...,...,...,...,...,...
momentum_pvo_hist,498.0,0.008003,4.869611,-10.896764,-3.222274,-0.563608,1.854736,21.877056
momentum_kama,498.0,2488.000450,115.010926,2171.543410,2397.578949,2488.057172,2575.407927,2773.873600
others_dr,498.0,0.014907,1.735466,-15.864134,-0.842570,0.040756,0.871331,6.019820
others_dlr,498.0,0.034309,1.584533,-8.133818,-0.837476,0.040747,0.867557,5.845587


In [9]:
data_stock.loc['BAJAJ-AUTO.NS'].describe()

,return,macd,signal,atr,natr,momentum,co,obv,mfi,hts,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
count,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000,4.980000e+02,4.980000e+02,498.000000,...,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000
mean,-0.060703,13.603356,13.214289,81.915661,2.157839,21.867829,-1464.163666,7.598501e+06,5.701480e+01,-0.000141,...,0.334985,0.334935,0.010505,-1.145273,-1.216227,-0.109326,3821.483124,0.055631,0.046617,0.367508
std,1.347858,55.423179,52.177601,17.092519,0.520363,154.438550,100398.074024,5.614458e+06,1.422514e-14,0.099914,...,1.434453,1.344027,0.441135,10.244379,7.052500,6.587461,376.556967,1.404542,1.403313,10.440160
min,-5.231660,-134.407518,-118.636531,48.446411,1.192395,-415.349854,-480382.278110,-3.166671e+06,5.701480e+01,-0.199466,...,-3.789158,-3.368783,-1.304788,-23.252670,-19.259819,-11.602282,3195.054897,-5.457210,-5.611765,-18.886161
25%,-0.822751,-17.125715,-14.594805,68.416940,1.776414,-68.437256,-59599.214143,2.827672e+06,5.701480e+01,-0.067956,...,-0.399513,-0.245770,-0.227780,-8.593604,-6.493803,-4.278654,3622.377812,-0.672857,-0.675131,-5.694909
50%,-0.086224,17.099795,12.682025,80.164280,2.114938,21.867829,-7032.835700,8.009706e+06,5.701480e+01,-0.022110,...,0.453004,0.341617,0.010505,-1.241184,-1.216227,-0.940610,3753.080262,0.035756,0.038932,-1.851392
75%,0.693644,49.205876,47.966979,91.193678,2.433700,118.287476,54541.576967,1.092821e+07,5.701480e+01,0.059144,...,1.285030,1.263042,0.296749,5.517813,3.313134,2.531253,3837.475642,0.785678,0.782608,2.158654
max,5.208739,142.330512,137.943757,130.396432,3.587814,480.800049,344595.402840,1.990074e+07,5.701480e+01,0.437997,...,3.405188,3.255042,1.230618,41.983369,18.200461,37.193411,4817.996416,5.900632,5.733104,28.817720


In [10]:
class Stock:
    name = None
    data = None
    scaler = None
    lstm_data =  None
    scaled_data = None
    def __init__(self, name, data):
        self.data = data
        self.name = name
        
    def prepare_lstm_data(self, lookback):
        X = []
        y = []
        for i in range(lookback, len(self.data)):
            X.append(self.data.iloc[i-lookback:i])
            y.append(self.data.iloc[i]["return"])
        X = np.array(X)
        y = np.array(y)

        self.lstm_data = {
            'X': X,
            'y': y
        }
        return self.lstm_data
    
    def prepare_scaled_data(self, scaler):
        self.scaler = scaler()
        self.scaled_data = self.scaler.fit_transform(self.data)

    def scale(self, scaler):
        self.scalerX = scaler()
        self.scalerY = scaler()

        shapeX = self.lstm_data['X'].shape
        reshaped_x = self.lstm_data['X'].reshape((shapeX[0]*shapeX[1], shapeX[2]))
        self.lstm_data['X'] = self.scalerX.fit_transform(reshaped_x).reshape(shapeX)

        shapeY = self.lstm_data['y'].shape
        reshaped_y = self.lstm_data['y'].reshape((shapeY[0], 1))
        self.lstm_data['y'] = self.scalerY.fit_transform(reshaped_y).reshape(shapeY[0])

        print(self.name, self.lstm_data['X'].shape, self.lstm_data['y'].shape) 

In [11]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

all_stocks = []
for stock in stocks:
    s = Stock(stock, data_stock.loc[stock])
    s.prepare_lstm_data(lookback=10)
    # s.scale(StandardScaler)
    s.scale(MinMaxScaler)
    all_stocks.append(s)

RELIANCE.NS (488, 10, 108) (488,)
TATASTEEL.NS (488, 10, 108) (488,)
HDFCBANK.NS (488, 10, 108) (488,)
INFY.NS (488, 10, 108) (488,)
BAJAJ-AUTO.NS (488, 10, 108) (488,)
ICICIBANK.NS (488, 10, 108) (488,)
ITC.NS (488, 10, 108) (488,)
UPL.NS (488, 10, 108) (488,)
ONGC.NS (488, 10, 108) (488,)
HINDALCO.NS (488, 10, 108) (488,)
TITAN.NS (488, 10, 108) (488,)
COALINDIA.NS (488, 10, 108) (488,)
INDUSINDBK.NS (488, 10, 108) (488,)
BAJAJFINSV.NS (488, 10, 108) (488,)
GRASIM.NS (488, 10, 108) (488,)
JSWSTEEL.NS (488, 10, 108) (488,)


In [12]:
# merge all stocks into one 
tX = []
ty = []
t = []
for stock in all_stocks:
    stock.prepare_scaled_data(MinMaxScaler)
    t.append(stock.scaled_data)

t = np.array(t)
t = t.reshape((t.shape[0]*t.shape[1], t.shape[2]))

t.shape

(7968, 108)

In [13]:
print(np.std(t, axis=0))

[0.13600592 0.21767636 0.22741927 0.21283963 0.21188151 0.16818066
 0.14076332 0.26509074 0.         0.20886496 0.25379187 0.2435454
 0.19045233 0.11651363 0.1996235  0.49975383 0.19495839 0.
 0.19495839 0.13590313 0.13560348 0.13520249 0.25513824 0.26650311
 0.20010471 0.21427185 0.15845689 0.1845087  0.21176245 0.2524672
 0.20849097 0.27972699 0.25863888 0.24546997 0.25410957 0.19670503
 0.19972893 0.25113673 0.21522917 0.25038054 0.25095817 0.25432719
 0.20351868 0.17753197 0.42721142 0.38416236 0.26329262 0.24837613
 0.25152658 0.20294822 0.30528871 0.16353016 0.2135373  0.21530073
 0.22429639 0.18379963 0.25260611 0.25924691 0.2540099  0.2602727
 0.20004007 0.19654266 0.20147689 0.21822193 0.19634179 0.16348729
 0.20707854 0.21113823 0.19586746 0.24384088 0.25194834 0.25143812
 0.2687552  0.40054346 0.20557199 0.17938487 0.18961108 0.17984512
 0.24819019 0.26802815 0.35409665 0.34972748 0.3142795  0.18418263
 0.17758666 0.21390841 0.21627867 0.20367362 0.35428792 0.32184004
 0.308

In [ ]:
from sklearn.decomposition import KernelPCA, PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

def get_pca(data, n_components=2):
    pca = PCA(n_components=n_components)
    pca.fit(data)
    return pca.transform(data)

def get_kpca(data, n_components=2, kernel='rbf'):
    kpca = KernelPCA(n_components=n_components, kernel=kernel, n_jobs=-1)
    kpca.fit(data)
    return kpca.transform(data)

def get_tsne(data, n_components=2, perplexity=30, n_iter=1000):

    tsne = TSNE(n_components=n_components, perplexity=perplexity, n_iter=n_iter, n_jobs=-1)
    return tsne.fit_transform(data)

kpcaData = get_kpca(t, n_components=50, kernel='rbf')
tsneData = get_tsne(kpcaData, n_components=2, perplexity=20, n_iter=2000)

# plot the data
plt.figure(figsize=(10,10))
plt.scatter(tsneData[:,0], tsneData[:,1])
plt.show()


In [ ]:
# save the data
np.save('tsneData.npy', tsneData)
np.save('kpcaData.npy', kpcaData)

In [16]:
for s in all_stocks:
    print(s.name,s.lstm_data['X'].shape,s.lstm_data['y'].shape)
    # break

RELIANCE.NS (488, 10, 108) (488,)
TATASTEEL.NS (488, 10, 108) (488,)
HDFCBANK.NS (488, 10, 108) (488,)
INFY.NS (488, 10, 108) (488,)
BAJAJ-AUTO.NS (488, 10, 108) (488,)
ICICIBANK.NS (488, 10, 108) (488,)
ITC.NS (488, 10, 108) (488,)
UPL.NS (488, 10, 108) (488,)
ONGC.NS (488, 10, 108) (488,)
HINDALCO.NS (488, 10, 108) (488,)
TITAN.NS (488, 10, 108) (488,)
COALINDIA.NS (488, 10, 108) (488,)
INDUSINDBK.NS (488, 10, 108) (488,)
BAJAJFINSV.NS (488, 10, 108) (488,)
GRASIM.NS (488, 10, 108) (488,)
JSWSTEEL.NS (488, 10, 108) (488,)


In [17]:
# combine all stocks
X = []
y = []
for stock in all_stocks:
    X.append(stock.lstm_data['X'])
    y.append(stock.lstm_data['y'])
X = np.concatenate(X)
y = np.concatenate(y)


In [18]:
X.shape, y.shape

((7808, 10, 108), (7808,))

In [19]:
# split train and test
from sklearn.model_selection import train_test_split
X_train = X[:int(len(X)*0.8)]
X_test = X[int(len(X)*0.8):]
y_train = y[:int(len(y)*0.8)]
y_test = y[int(len(y)*0.8):]

X_train.shape, X_test.shape, y_train.shape, y_test.shape


((6246, 10, 108), (1562, 10, 108), (6246,), (1562,))

In [20]:
#Define the LSTM model
model_stock = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True,input_shape=(X_train.shape[1], X_train.shape[2]))),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1)
])
# Compile the model
model_stock.compile(optimizer='adam', loss='mse')

In [21]:
history = model_stock.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1, shuffle=False)

Epoch 1/10
176/176 [==============================] - 7s 22ms/step - loss: 0.0381 - val_loss: 0.0201
Epoch 2/10
176/176 [==============================] - 4s 20ms/step - loss: 0.0178 - val_loss: 0.0215
Epoch 3/10
176/176 [==============================] - 5s 28ms/step - loss: 0.0176 - val_loss: 0.0236
Epoch 4/10
176/176 [==============================] - 5s 27ms/step - loss: 0.0174 - val_loss: 0.0231
Epoch 5/10
176/176 [==============================] - 5s 29ms/step - loss: 0.0181 - val_loss: 0.0203
Epoch 6/10
176/176 [==============================] - 5s 28ms/step - loss: 0.0179 - val_loss: 0.0213
Epoch 7/10
176/176 [==============================] - 5s 30ms/step - loss: 0.0181 - val_loss: 0.0220
Epoch 8/10
176/176 [==============================] - 5s 27ms/step - loss: 0.0184 - val_loss: 0.0221
Epoch 9/10
176/176 [==============================] - 5s 26ms/step - loss: 0.0180 - val_loss: 0.0215
Epoch 10/10
176/176 [==============================] - 4s 25ms/step - loss: 0.0178 - val_lo

In [ ]:
model_stock.summary()

In [26]:
# Write a data loader for the dataset

import torch
# get dataloader
from torch.utils.data import DataLoader, TensorDataset

dataset = [X_train, y_train, X_test, y_test]
train_dataset = TensorDataset(torch.tensor(dataset[0]), torch.tensor(dataset[1]))
test_dataset = TensorDataset(torch.tensor(dataset[2]), torch.tensor(dataset[3]))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)


In [31]:
!which python

/Users/harshit/miniforge3/bin/python


In [30]:
train_dataset[0]

AttributeError: 'tuple' object has no attribute 'shape'

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


# create same model in pytorch
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.LSTM(input_size = (X_train.shape[1],X_train.shape[2]), hidden_size=128, bidirectional=True, batch_first=True),
            nn.LSTM(64, bidirectional=True, batch_first=True),
            nn.Dropout(0.1),
            nn.Linear(128,1)
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [25]:
model = Net()
# model no.of parameters
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

TypeError: RNNBase.__init__() missing 1 required positional argument: 'hidden_size'

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
# test the model
model_stock.evaluate(X_test, y_test)

In [ ]:
y_pred = model_stock.predict(X_test)
y_pred = stock.scalerY.inverse_transform(y_pred)
y_true = stock.scalerY.inverse_transform(y_test.reshape(-1, 1))


In [ ]:
# plot error
plt.plot(y_true, label='true')
plt.plot(y_pred, label='pred')
# plt.plot(y_true-y_pred, label='error')
plt.legend()
# increase plot width
plt.rcParams["figure.figsize"] = (30,3)
plt.show()

In [ ]:
y_pred = model_stock.predict(X_test)

In [ ]:
error = y_true-y_pred
plt.hist(error, bins=100)
plt.show()

In [ ]:
# plot error
plt.plot(y_true, label='true')
plt.plot(y_pred, label='pred')

    

In [ ]:
capital=100000
n_stocks=5
portfolio_value=capital
import math
for i in range(len(y_test_stock[1])+1):
    if(i==len(y_test_stock[1])):
      portfolio_value=100000
      capital=portfolio_value
      print("Day {0} ".format(i+1))
      predicted_returns=[]
      for j in range(len(y_test_stock)):
        predicted_returns.append(p_stock[j][i])
      predicted_returns=np.array(predicted_returns)
      predicted_returns=np.squeeze(predicted_returns,axis=1)
      indices = np.argsort(np.abs(predicted_returns))[::-1]
      predicted_returns = predicted_returns[indices]
    # print(returns)

    # returns=np.nan_to_num(returns)
    # print("returns actual",returns)

      p=[]
      # r=[]
      count=0
      for j in range(len(predicted_returns)):
        if(count==n_stocks):
          break
        # if(np.isnan(returns[j])):
        #   continue
        else:
          p.append(predicted_returns[j])
          # r.append(returns[j])
          count+=1

      p=np.array(p)
      weights = (p) / np.sum(np.abs(p))
    # print("weights new",weights)

      capital_each=portfolio_value*weights
      for a in range(count):
        print(x[indices[a]],end=" ")
      print()
      print("capital_each. =", capital_each)
      # print(sum(abs(capital_each)))
      print("predictions= ",p)
      # print("Day {0} ".format(i+1))
      continue

    # if (i==79):
    #   capital=97547
    #   portfolio_value=capital
    predicted_returns=[]
    for j in range(len(y_test_stock)):
      predicted_returns.append(p_stock[j][i])
    # for j in range(len(y_test_crypto)):
    #   predicted_returns.append(p_crypto[j][i])
    # for j in range(len(y_test_commodity)):
    #   predicted_returns.append(p_commodity[j][i])

    predicted_returns=np.array(predicted_returns)
    # print("predicted_returns",predicted_returns)
    returns=[]
    for j in range(len(y_test_stock)):
      print(i,j)
      returns.append(y_test_stock[j][i])

    # for j in range(len(y_test_crypto)):
    #   returns.append(y_test_crypto[j][i])
    # for j in range(len(y_test_commodity)):
    #   returns.append(y_test_commodity[j][i])
    # print("returns actual",returns)
    predicted_returns=np.array(predicted_returns)
    returns=np.array(returns)
    # print(predicted_returns.shape)
    # print(returns.shape)
    predicted_returns=np.squeeze(predicted_returns,axis=1)
    indices = np.argsort(np.abs(predicted_returns))[::-1]
    # print("indices= ",indices)
    # indices=indices[0]


    # Sort both 'a' and 'b' arrays in the same order
    returns = returns[indices]
    predicted_returns = predicted_returns[indices]
    # print(returns)

    # returns=np.nan_to_num(returns)
    # print("returns actual",returns)

    p=[]
    r=[]
    count=0
    for j in range(len(predicted_returns)):
      if(count==n_stocks):
        break
      if(np.isnan(returns[j])):
        continue
      else:
        p.append(predicted_returns[j])
        r.append(returns[j])
        count+=1

    p=np.array(p)
    r=np.array(r)
    # print(p.shape)
    print(p)
    print(r)
    # print(r.shape)
    # print("pred retuns",p)
    # print("actual retuns",r)
    weights = (p) / np.sum(np.abs(p))
    # print("weights new",weights)

    capital_each=portfolio_value*weights
    cap_diff=0
    not_stock=[-1]
    for idx in range(n_stocks):
      if indices[idx]<len(x):
        print(indices[idx])
        capital_new=math.floor(capital_each[idx]/open_prices_stock_t[i][indices[idx]])*open_prices_stock_t[i][indices[idx]]
        print("Quantity of Stock "+ x[indices[idx]]+":"+ str(capital_new/open_prices_stock_t[i][indices[idx]]))
        cap_diff+=capital_each[idx]-capital_new
        capital_each[idx]=capital_new
      else:
        not_stock.append(idx)
    val=0

    # if len(not_stock)==1:
    #   val+=cap_diff
    # else:
    #   capital_each[not_stock[1]]+=cap_diff

    print(capital_each)
    for a in range(count):
      print(x[indices[a]],end=" ")
    print()
    print(capital_each.shape)
    print(sum(abs(capital_each)))
    # print("capital_each",capital_each)
    gain_loss=np.dot(capital_each,r)
    print("Day {0} return is {1}".format(i+1,gain_loss/portfolio_value*100))
    # print("gain loss",gain_loss)
    portfolio_value += (gain_loss+val -400)

    # capital = portfolio_value
    print("gain/loss:",gain_loss,end=", ")
    print("portfolio_value:",portfolio_value)


# Print the final portfolio value
print("Final portfolio value:", portfolio_value)

